In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
WORKING_DIR = os.path.dirname(os.path.abspath(''))
sys.path.insert(1, os.path.join(WORKING_DIR,'LensQuEst'))

#to get latex to work, shoulldn't be necessary for most ppl
os.environ['PATH'] = "%s:/usr/local/cuda-11.2/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/opt/anaconda/bin:/home/delon/texlive/bin/x86_64-linux:/home/delon/.local/bin:/home/delon/bin"%os.environ['PATH']

In [3]:
from universe import *
from halo_fit import *
from cmb import *
from flat_map import *
from weight import *
from pn_2d import *
import pickle
import seaborn as sns
from scipy.stats import spearmanr


In [4]:
print("Map properties")

# number of pixels for the flat map
nX = 800 # 1200
nY = 800 #1200

# map dimensions in degrees
sizeX = 20.
sizeY = 20.

# basic map object
baseMap = FlatMap(nX=nX, nY=nY, sizeX=sizeX*np.pi/180., sizeY=sizeY*np.pi/180.)

# multipoles to include in the lensing reconstruction
lMin = 30.; lMax = 3.5e3

# ell bins for power spectra
nBins = 21  # number of bins
lRange = (1., 2.*lMax)  # range for power spectra

Map properties


In [5]:
print("CMB experiment properties")

# Adjust the lMin and lMax to the assumptions of the analysis
# CMB S3 specs
cmb = StageIVCMB(beam=1.4, noise=7., lMin=lMin, lMaxT=lMax, lMaxP=lMax, atm=False)

# Total power spectrum, for the lens reconstruction
# basiscally gets what we theoretically expect the
# power spectrum will look like
forCtotal = lambda l: cmb.ftotal(l) 

# reinterpolate: gain factor 10 in speed
L = np.logspace(np.log10(lMin/2.), np.log10(2.*lMax), 1001, 10.)
F = np.array(list(map(forCtotal, L)))
cmb.fCtotal = interp1d(L, F, kind='linear', bounds_error=False, fill_value=0.)

CMB experiment properties


In [6]:
print("CMB lensing power spectrum")
u = UnivPlanck15()
halofit = Halofit(u, save=False)
w_cmblens = WeightLensSingle(u, z_source=1100., name="cmblens")
p2d_cmblens = P2dAuto(u, halofit, w_cmblens, save=False)

CMB lensing power spectrum


In [7]:
print("Gets a theoretical prediction for the noise")
fNqCmb_fft = baseMap.forecastN0Kappa(cmb.funlensedTT, cmb.fCtotal, lMin=lMin, lMax=lMax, test=False)
Ntheory = lambda l: fNqCmb_fft(l) 

Gets a theoretical prediction for the noise
computing the reconstruction noise


/home/users/delon/LensQuEst/LensQuEst/flat_map.py:1668: RuntimeWarning: divide by zero encountered in divide
  resultFourier = 1./resultFourier
/home/users/delon/LensQuEst/LensQuEst/flat_map.py:1668: RuntimeWarning: invalid value encountered in divide
  resultFourier = 1./resultFourier


In [8]:
from tqdm import tqdm, trange

In [9]:
in_data = {}
IN_DATA_FNAMES = ['/oak/stanford/orgs/kipac/users/delon/LensQuEst/map_sims_800x800_20x20_%d.pkl'%(i) for i in range(1,51)]

for fname in tqdm(IN_DATA_FNAMES):
    f = open(fname, 'rb') 
    c_in_data = pickle.load(f) 
    f.close()
    for key in c_in_data:
        if(key != 'kCmbF_1' and key != 'lCmbF_1' and key!='cmb0F_1' and key!='noiseF_1' and key!='fgF_1'):
            continue
        if(key not in in_data.keys()):
            in_data[key] = np.array(c_in_data[key])
        else:
            in_data[key] = np.vstack( (in_data[key],np.array(c_in_data[key])) )


for key in in_data:
    print(key, np.shape(in_data[key]))


100%|██████████| 50/50 [02:14<00:00,  2.69s/it]

cmb0F_1 (500, 800, 401)
kCmbF_1 (500, 800, 401)
lCmbF_1 (500, 800, 401)
fgF_1 (500, 800, 401)
noiseF_1 (500, 800, 401)


In [10]:
# data = {}
# pairs = [
# #    [0,0], #N0
# #    [0,1], #kappa
# #    [1,0], #kappa
# #    [0,2], #N1
# #    [1,1], #N1
# #    [2,0], #N1
# #     [0,3], #should vanish
# #     [1,2], #should vanish
# #     [2,1], #should vanish
# #     [3,0], #should vanish
# #     [0,4], #N2 
# #     [1,3], #N2
# #     [2,2], #N2
# #     [3,1], #N2
# #     [4,0], #N2
#    [-1, -1], #QE
#    [-2, -2], #unlensed
# ]


# data_names = {
#     0: 'cmb0F_1',
#     1: 'lCmbF_o1_1',
#     2: 'lCmbF_o2_1',
#     3: 'lCmbF_o3_1',
#     4: 'lCmbF_o4_1',
#     -1: 'lCmbF_1',
#     -2: 'totalF_0',
# }
# # pbar = trange(len(pairs))
# for file_idx in trange(1,51):
#     for pair_idx in range(len(pairs)):
#         pair = pairs[pair_idx]
#         c_fname = '/oak/stanford/orgs/kipac/users/delon/LensQuEst/QE_and_Nhat_from_map_sims_800x800_20x20_Clunlensed_weight_FILE%d_pair_%d_%d.pkl'%(file_idx, pair[0], pair[1])
#         if(not os.path.isfile(c_fname)):
#             continue
#         f = open(c_fname, 'rb')  
#         c_data = pickle.load(f)
#         f.close()        
#         for key in c_data:
#             if(key not in data.keys()):
#                 data[key] = np.array(c_data[key])
#             else:
#                 data[key] = np.vstack((data[key], np.array(c_data[key])))  
# #             print(np.shape(data[key]))

In [11]:
for key in in_data:
    print(key)

cmb0F_1
kCmbF_1
lCmbF_1
fgF_1
noiseF_1


In [12]:
# from tqdm import trange
# ck = 'kkT'
# from multiprocessing import Pool


# ps_data = {}
# # for i in trange(len(in_data['kCmbF_1'])):
# for i in trange(10):
#     kCmbFourier = in_data['kCmbF_1'][i]

#     Cls = {}
    
#     dataF0 = in_data['lCmbF_1'][i]
#     dataF1 = dataF0
    
#     kap = baseMap.computeQuadEstKappaNorm(cmb.funlensedTT, cmb.fCtotal, 
#                                          lMin=lMin, lMax=lMax, 
#                                          dataFourier=dataF0,
#                                          dataFourier2=dataF1)
        
#     Cls['kkT'] = [0,0,0]
#     Cls['kkT'][0],Cls['kkT'][1],Cls['kkT'][2] = baseMap.crossPowerSpectrum(dataFourier1=kap, 
#                                                                            dataFourier2=kCmbFourier)
#     a,b,c = baseMap.powerSpectrum(kCmbFourier)
    
#     assert((a == Cls['kkT'][0]).all())
#     Cls['kkT'][1]-=b
#     Cls['kkT'][1]/=b
#     Cls['kkT'][2]/=b
        
#     if(ck not in ps_data.keys()):
#         ps_data[ck] = np.array([Cls[ck]])
#     else:
#         ps_data[ck] = np.vstack((ps_data[ck], np.array([Cls[ck]])))  
    

In [13]:
from multiprocessing import Pool
from tqdm import tqdm
ps_data = {}
ck = 'kkT'

def compute_cls(i):
    kCmbFourier = in_data['kCmbF_1'][i]
    Cls = {}

    cmbo0_F = in_data['cmb0F_1'][i]
    
    cmbo0 = baseMap.inverseFourier(cmbo0_F)
    cmbo1 = baseMap.doLensingTaylor(unlensed=cmbo0, kappaFourier=kCmbFourier, order=1)-cmbo0
    cmbo1_F = baseMap.fourier(cmbo1)

    
    dataFo0 = cmbo0_F + in_data['fgF_1'][i] + in_data['noiseF_1'][i]
    dataFo1 = cmbo1_F + in_data['fgF_1'][i] + in_data['noiseF_1'][i]
    
    
    
    QE01 = baseMap.computeQuadEstKappaNorm(cmb.funlensedTT, cmb.fCtotal, 
                                         lMin=lMin, lMax=lMax, 
                                         dataFourier=dataFo0,
                                         dataFourier2=dataFo1)
    QE10 = baseMap.computeQuadEstKappaNorm(cmb.funlensedTT, cmb.fCtotal, 
                                         lMin=lMin, lMax=lMax, 
                                         dataFourier=dataFo1,
                                         dataFourier2=dataFo0)

    Cls['kkT'] = [0, 0, 0]
    Cls['kkT'][0], Cls['kkT'][1], Cls['kkT'][2] = baseMap.crossPowerSpectrum(dataFourier1=QE01 + QE10,
                                                                              dataFourier2=kCmbFourier)
    a, b, c = baseMap.powerSpectrum(kCmbFourier)

    assert((a == Cls['kkT'][0]).all())
    Cls['kkT'][1] -= b
    Cls['kkT'][1] /= b
    Cls['kkT'][2] /= b

    return Cls

N_runs = len(in_data['kCmbF_1'])
# N_runs = 10
with Pool() as pool:
    results = list(tqdm(pool.imap(compute_cls, range(N_runs)), total=(N_runs)))

for i, result in enumerate(results):
    ck = 'kkT'
    if ck not in ps_data.keys():
        ps_data[ck] = np.array([result[ck]])
    else:
        ps_data[ck] = np.vstack((ps_data[ck], np.array([result[ck]])))


  0%|          | 0/500 [00:00<?, ?it/s]/home/users/delon/LensQuEst/LensQuEst/flat_map.py:1208: RuntimeWarning: invalid value encountered in divide
  dxFourier = -2.j * self.lx/self.l**2 * kappaFourier
/home/users/delon/LensQuEst/LensQuEst/flat_map.py:1208: RuntimeWarning: invalid value encountered in divide
  dxFourier = -2.j * self.lx/self.l**2 * kappaFourier
/home/users/delon/LensQuEst/LensQuEst/flat_map.py:1208: RuntimeWarning: invalid value encountered in divide
  dxFourier = -2.j * self.lx/self.l**2 * kappaFourier
/home/users/delon/LensQuEst/LensQuEst/flat_map.py:1208: RuntimeWarning: invalid value encountered in divide
  dxFourier = -2.j * self.lx/self.l**2 * kappaFourier
/home/users/delon/LensQuEst/LensQuEst/flat_map.py:1208: RuntimeWarning: invalid value encountered in divide
  dxFourier = -2.j * self.lx/self.l**2 * kappaFourier
/home/users/delon/LensQuEst/LensQuEst/flat_map.py:1208: RuntimeWarning: invalid value encountered in divide
  dxFourier = -2.j * self.lx/self.l**2 * ka

In [14]:
print(1)

In [15]:
def combine_Cl(Cls_tot):
    n_runs = np.shape(Cls_tot)[0]
    print(n_runs, np.shape(Cls_tot))
    lCen = Cls_tot[0][0]
    Cls = np.sum(np.transpose(Cls_tot, axes=[1,2,0])[1], axis=1)/n_runs
    sCls = np.sqrt(np.sum(np.square(np.transpose(Cls_tot, axes=[1,2,0])[2]), axis=1))/n_runs
    return lCen, Cls, sCls

In [19]:
plt.rcParams.update({'font.size': 30})

In [25]:
fig,ax = plt.subplots(nrows=1, figsize=(20,16))

lCen, Cl, sCl = combine_Cl(np.array(ps_data[ck]))
ax.fill_between(x=[0,1e4], y1=[-.1, -.1], y2=[.1,.1], color='lightgrey', label='10\% error')
ax.fill_between(x=[0,1e4], y1=[-.01, -.01], y2=[.01,.01], color='grey', label='1\% error')


ax.errorbar(lCen, (Cl), yerr=sCl, alpha=.75, 
            fmt=':', capsize=3, capthick=1)


tmp_data = {
        'x': lCen,
        'y1': [y - e for y, e in zip((Cl), sCl)],
        'y2': [y + e for y, e in zip((Cl), sCl)]}
ax.fill_between(**tmp_data,  alpha=.25)

plt.xscale('log')
plt.xlabel(r'$\ell$')


plt.yscale('symlog', linthresh=.01)

plt.ylabel(r'$\frac{\left<(QE[T^0,T^1]+QE[T^1,T^0]) \kappa\right>-\left<\kappa \kappa\right>}{\left<\kappa \kappa\right>}$')
plt.ylim(-1,1)
plt.xlim(30,1e4)

plt.axhline(0, linewidth=1, linestyle="--", color="black")
plt.title(r'$QE$ with $C_\ell^{\rm unlensed}$ weights')
plt.legend(frameon=False)
plt.savefig('figures/check_norm_unlensed_weights.pdf', bbox_inches='tight')

In [28]:

from multiprocessing import Pool
from tqdm import tqdm
ps_data = {}
ck = 'kkT_lw'

def compute_cls(i):
    kCmbFourier = in_data['kCmbF_1'][i]
    Cls = {}

    cmbo0_F = in_data['cmb0F_1'][i]
    
    cmbo0 = baseMap.inverseFourier(cmbo0_F)
    cmbo1 = baseMap.doLensingTaylor(unlensed=cmbo0, kappaFourier=kCmbFourier, order=1)-cmbo0
    cmbo1_F = baseMap.fourier(cmbo1)

    
    dataFo0 = cmbo0_F + in_data['fgF_1'][i] + in_data['noiseF_1'][i]
    dataFo1 = cmbo1_F + in_data['fgF_1'][i] + in_data['noiseF_1'][i]
    
    
    
    QE01 = baseMap.computeQuadEstKappaNormLensedWeights(cmb.funlensedTT, cmb.flensedTT, cmb.fCtotal, 
                                         lMin=lMin, lMax=lMax, 
                                         dataFourier=dataFo0,
                                         dataFourier2=dataFo1)
    QE10 = baseMap.computeQuadEstKappaNormLensedWeights(cmb.funlensedTT, cmb.flensedTT, cmb.fCtotal, 
                                         lMin=lMin, lMax=lMax, 
                                         dataFourier=dataFo1,
                                         dataFourier2=dataFo0)

    Cls['kkT_lw'] = [0, 0, 0]
    Cls['kkT_lw'][0], Cls['kkT_lw'][1], Cls['kkT_lw'][2] = baseMap.crossPowerSpectrum(dataFourier1=QE01 + QE10,
                                                                              dataFourier2=kCmbFourier)
    a, b, c = baseMap.powerSpectrum(kCmbFourier)

    assert((a == Cls['kkT_lw'][0]).all())
    Cls['kkT_lw'][1] -= b
    Cls['kkT_lw'][1] /= b
    Cls['kkT_lw'][2] /= b

    return Cls

N_runs = len(in_data['kCmbF_1'])
# N_runs = 10
with Pool() as pool:
    results = list(tqdm(pool.imap(compute_cls, range(N_runs)), total=(N_runs)))

for i, result in enumerate(results):
    ck = 'kkT_lw'
    if ck not in ps_data.keys():
        ps_data[ck] = np.array([result[ck]])
    else:
        ps_data[ck] = np.vstack((ps_data[ck], np.array([result[ck]])))

In [29]:
fig,ax = plt.subplots(nrows=1, figsize=(20,16))

lCen, Cl, sCl = combine_Cl(np.array(ps_data[ck]))
ax.fill_between(x=[0,1e4], y1=[-.1, -.1], y2=[.1,.1], color='lightgrey', label='10\% error')
ax.fill_between(x=[0,1e4], y1=[-.01, -.01], y2=[.01,.01], color='grey', label='1\% error')


ax.errorbar(lCen, (Cl), yerr=sCl, alpha=.75, 
            fmt=':', capsize=3, capthick=1)


tmp_data = {
        'x': lCen,
        'y1': [y - e for y, e in zip((Cl), sCl)],
        'y2': [y + e for y, e in zip((Cl), sCl)]}
ax.fill_between(**tmp_data,  alpha=.25)

plt.xscale('log')
plt.xlabel(r'$\ell$')


plt.yscale('symlog', linthresh=.01)

plt.ylabel(r'$\frac{\left<(QE[T^0,T^1]+QE[T^1,T^0]) \kappa\right>-\left<\kappa \kappa\right>}{\left<\kappa \kappa\right>}$')
plt.ylim(-1,1)
plt.xlim(30,1e4)

plt.axhline(0, linewidth=1, linestyle="--", color="black")
plt.title(r'$QE$ with $C_\ell^{\rm lensed}$ weights')
plt.legend(frameon=False)
plt.savefig('figures/check_norm_lensed_weights.pdf', bbox_inches='tight')

In [30]:

from multiprocessing import Pool
from tqdm import tqdm
ps_data = {}
ck = 'kkT_lw_wrong'

def compute_cls(i):
    kCmbFourier = in_data['kCmbF_1'][i]
    Cls = {}

    cmbo0_F = in_data['cmb0F_1'][i]
    
    cmbo0 = baseMap.inverseFourier(cmbo0_F)
    cmbo1 = baseMap.doLensingTaylor(unlensed=cmbo0, kappaFourier=kCmbFourier, order=1)-cmbo0
    cmbo1_F = baseMap.fourier(cmbo1)

    
    dataFo0 = cmbo0_F + in_data['fgF_1'][i] + in_data['noiseF_1'][i]
    dataFo1 = cmbo1_F + in_data['fgF_1'][i] + in_data['noiseF_1'][i]
    
    
    
    QE01 = baseMap.computeQuadEstKappaNorm(cmb.flensedTT, cmb.fCtotal,
                                         lMin=lMin, lMax=lMax, 
                                         dataFourier=dataFo0,
                                         dataFourier2=dataFo1)
    QE10 = baseMap.computeQuadEstKappaNorm(cmb.flensedTT, cmb.fCtotal,
                                         lMin=lMin, lMax=lMax, 
                                         dataFourier=dataFo1,
                                         dataFourier2=dataFo0)

    Cls['kkT_lw_wrong'] = [0, 0, 0]
    Cls['kkT_lw_wrong'][0], Cls['kkT_lw_wrong'][1], Cls['kkT_lw_wrong'][2] = baseMap.crossPowerSpectrum(dataFourier1=QE01 + QE10,
                                                                              dataFourier2=kCmbFourier)
    a, b, c = baseMap.powerSpectrum(kCmbFourier)

    assert((a == Cls['kkT_lw_wrong'][0]).all())
    Cls['kkT_lw_wrong'][1] -= b
    Cls['kkT_lw_wrong'][1] /= b
    Cls['kkT_lw_wrong'][2] /= b

    return Cls

N_runs = len(in_data['kCmbF_1'])
# N_runs = 10
with Pool() as pool:
    results = list(tqdm(pool.imap(compute_cls, range(N_runs)), total=(N_runs)))

for i, result in enumerate(results):
    ck = 'kkT_lw_wrong'
    if ck not in ps_data.keys():
        ps_data[ck] = np.array([result[ck]])
    else:
        ps_data[ck] = np.vstack((ps_data[ck], np.array([result[ck]])))

In [31]:
fig,ax = plt.subplots(nrows=1, figsize=(20,16))

lCen, Cl, sCl = combine_Cl(np.array(ps_data[ck]))
ax.fill_between(x=[0,1e4], y1=[-.1, -.1], y2=[.1,.1], color='lightgrey', label='10\% error')
ax.fill_between(x=[0,1e4], y1=[-.01, -.01], y2=[.01,.01], color='grey', label='1\% error')


ax.errorbar(lCen, (Cl), yerr=sCl, alpha=.75, 
            fmt=':', capsize=3, capthick=1)


tmp_data = {
        'x': lCen,
        'y1': [y - e for y, e in zip((Cl), sCl)],
        'y2': [y + e for y, e in zip((Cl), sCl)]}
ax.fill_between(**tmp_data,  alpha=.25)

plt.xscale('log')
plt.xlabel(r'$\ell$')


plt.yscale('symlog', linthresh=.01)

plt.ylabel(r'$\frac{\left<(QE[T^0,T^1]+QE[T^1,T^0]) \kappa\right>-\left<\kappa \kappa\right>}{\left<\kappa \kappa\right>}$')
plt.ylim(-1,1)
plt.xlim(30,1e4)

plt.axhline(0, linewidth=1, linestyle="--", color="black")
plt.title(r'$QE$ with $C_\ell^{\rm lensed}$ weights but old implementation (should have wrong norm)')
plt.legend(frameon=False)
plt.savefig('figures/check_norm_lensed_weights_old_implementation.pdf', bbox_inches='tight')